<a href="https://colab.research.google.com/github/petersonrs/projetostack/blob/main/Koalas01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Origem do estudo:
https://www.kaggle.com/thiagopanini/e-commerce-sentiment-analysis-eda-viz-nlp/notebook

# Caminhos dos dados

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
clientes = '/content/drive/MyDrive/Colab Notebooks/olist_customers_dataset.csv'
lista_pedidos = '/content/drive/MyDrive/Colab Notebooks/olist_order_items_dataset.csv'
pgto_pedidos = '/content/drive/MyDrive/Colab Notebooks/olist_order_payments_dataset.csv'
avaliacoes_pedidos = '/content/drive/MyDrive/Colab Notebooks/olist_order_reviews_dataset.csv'
pedidos = '/content/drive/MyDrive/Colab Notebooks/olist_orders_dataset.csv'
produtos = '/content/drive/MyDrive/Colab Notebooks/olist_products_dataset.csv'
vendedores = '/content/drive/MyDrive/Colab Notebooks/olist_sellers_dataset.csv'
categoria_produtos = '/content/drive/MyDrive/Colab Notebooks/product_category_name_translation.csv'

# Caminho dos arquivos do Peterson

In [45]:
clientes = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_customers_dataset.csv'
lista_pedidos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_order_items_dataset.csv'
pgto_pedidos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_order_payments_dataset.csv'
avaliacoes_pedidos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_order_reviews_dataset.csv'
pedidos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_orders_dataset.csv'
produtos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_products_dataset.csv'
vendedores = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_sellers_dataset.csv'
categoria_produtos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/product_category_name_translation.csv'

# Import libs

In [2]:
import pandas as pd
import numpy as np

# lendo os csvs

In [47]:
ds_lista_pedidos = pd.read_csv(lista_pedidos, index_col=False, encoding='utf8')
ds_pgto_pedidos = pd.read_csv(pgto_pedidos, index_col=False, encoding='utf8')
ds_avaliacoes_pedidos = pd.read_csv(avaliacoes_pedidos, index_col=False, encoding='utf8')
ds_pedidos = pd.read_csv(pedidos, index_col=False, encoding='utf8')
ds_produtos = pd.read_csv(produtos, index_col=False, encoding='utf8')
ds_vendedores = pd.read_csv(vendedores, index_col=False, encoding='utf8')
ds_clientes = pd.read_csv(clientes, index_col=False, encoding='utf8')
ds_categoria_produtos = pd.read_csv(categoria_produtos, index_col=False, encoding='utf8')

## join entre as tabelas para montar um tabelão

In [48]:
result = ds_pedidos.merge(ds_lista_pedidos, on="order_id", how="left")

In [49]:
result = result.merge(ds_produtos, on='product_id', how='left')

In [50]:
result = result.merge(ds_vendedores, on='seller_id', how='left')

In [51]:
result = result.merge(ds_clientes, on='customer_id', how='left')

In [52]:
result = result.merge(ds_avaliacoes_pedidos, on='order_id', how='left')

In [53]:
result = result.merge(ds_pgto_pedidos, on='order_id', how='left')

## exibindo como ficaram junções

In [54]:
result.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1.0,credit_card,1.0,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,3.0,voucher,1.0,2.00
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,2.0,voucher,1.0,18.59
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,1.0,boleto,1.0,141.46
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,1.0,credit_card,3.0,179.12


## backup do resultado e store na pasta de dados processados

In [55]:
transformacao_final = result.copy()

In [ ]:
transformacao_final.to_csv('/content/drive/MyDrive/Colab Notebooks/dados_transformados.csv',  index=False, encoding='utf8')

## Copia do arquivo processado na sessao do Peterson

In [56]:
transformacao_final.to_csv('/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados.csv',  index=False, encoding='utf8')

In [57]:
transformacao_final.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1.0,credit_card,1.0,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,3.0,voucher,1.0,2.00


In [ ]:
## Transformando *.csv em *.parquet
transformacao_final.to_parquet('/content/drive/MyDrive/Colab Notebooks/dados_transformados.parquet', index=False)

In [ ]:
#transformacao_final = transformacao_final.drop(columns=['review_comment_title'], axis=1)
#transformacao_final.head(5)

## Testando review_score
## 1-2 Ruim ou Péssimo
## 4-5 Bom ou Ótimo
## 3 Neutro

Adicionando uma coluna nova para receber novos valores de titulo

In [97]:
transformacao_final['new_title_review'] = None

In [98]:
def TestReviewScoreTitle():
  for index, row in transformacao_final.iterrows():
    if (transformacao_final['review_score'][index] >= 4 and transformacao_final['review_comment_title'][index] is np.nan):
        transformacao_final['new_title_review'][index] = 'bom ou otimo'
    elif (transformacao_final['review_score'][index] < 3 and transformacao_final['review_comment_title'][index] is np.nan):
          transformacao_final['new_title_review'][index] = 'ruim ou pessimo'
    elif (transformacao_final['review_score'][index] == 3 and transformacao_final['review_comment_title'][index] is np.nan):
          transformacao_final['new_title_review'][index] = 'neutro'
    else:
      transformacao_final['new_title_review'][index] = transformacao_final['review_comment_title'][index]


In [99]:
TestReviewScoreTitle()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/py

In [103]:
transformacao_final.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value,new_title_review,new_message_review
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1.0,credit_card,1.0,18.12,bom ou otimo,None
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,3.0,voucher,1.0,2.00,bom ou otimo,None
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,2.0,voucher,1.0,18.59,bom ou otimo,None
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,1.0,boleto,1.0,141.46,Muito boa a loja,None
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,1.0,credit_card,3.0,179.12,bom ou otimo,None


## Ajustando as mensagens nulas

In [104]:
transformacao_final['new_message_review'] = None

In [105]:
def TestReviewScoreMessages():
  for index, row in transformacao_final.iterrows():
    if (transformacao_final['review_score'][index] >= 4 and transformacao_final['review_comment_message'][index] is np.nan):
        transformacao_final['new_message_review'][index] = 'bom ou otimo'
    elif (transformacao_final['review_score'][index] < 3 and transformacao_final['review_comment_message'][index] is np.nan):
        transformacao_final['new_message_review'][index] = 'ruim ou pessimo'
    elif (transformacao_final['review_score'][index] == 3 and transformacao_final['review_comment_message'][index] is np.nan):
        transformacao_final['new_message_review'][index] = 'neutro'
    else:
      transformacao_final['new_message_review'][index] = transformacao_final['review_comment_message'][index]

In [ ]:
TestReviewScoreMessages()

In [ ]:
transformacao_final.head(50)

# Salvando arquivo com textos processados.
Agora pode ler a partir deste arquivo.

In [108]:
transformacao_final.to_csv('/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_tratado.csv',  index=False, encoding='utf8')

#A partir daqui Novos tratamentos no arquivo
#Tratar parte nlp nos titulos e mensagens

In [109]:
dados_tratados = '/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_tratado.csv'
ds_dados_tratados = pd.read_csv(dados_tratados, index_col=False, encoding='utf8')
transformacao_final = ds_dados_tratados.copy() # só pra manter o mesmo nome


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [111]:
transformacao_final.head(4)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value,new_title_review,new_message_review
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149.0,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1.0,credit_card,1.0,18.12,bom ou otimo,"Não testei o produto ainda, mas ele veio corre..."
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149.0,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,3.0,voucher,1.0,2.00,bom ou otimo,"Não testei o produto ainda, mas ele veio corre..."
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149.0,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,2.0,voucher,1.0,18.59,bom ou otimo,"Não testei o produto ainda, mas ele veio corre..."
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,af07308b275d755c9edb36a90c618231,47813.0,barreiras,BA,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,1.0,boleto,1.0,141.46,Muito boa a loja,Muito bom o produto.


## tratando apenas reviews

In [113]:
df_titulos_mensagens = transformacao_final[['new_message_review', 'new_title_review','order_id']]

In [116]:
df_titulos_mensagens

,new_message_review,new_title_review,order_id
0,"Não testei o produto ainda, mas ele veio corre...",bom ou otimo,e481f51cbdc54678b7cc49136f2d6af7
1,"Não testei o produto ainda, mas ele veio corre...",bom ou otimo,e481f51cbdc54678b7cc49136f2d6af7
2,"Não testei o produto ainda, mas ele veio corre...",bom ou otimo,e481f51cbdc54678b7cc49136f2d6af7
3,Muito bom o produto.,Muito boa a loja,53cdb2fc8bc7dce0b6741e2150273451
4,bom ou otimo,bom ou otimo,47770eb9100c2d0c44946d9cf07ec65d
...,...,...,...
119149,So uma peça que veio rachado mas tudo bem rs,bom ou otimo,63943bddc261676b46f01ca7ac2f7bd8
119150,Foi entregue antes do prazo.,bom ou otimo,83c1379a015df1e13d02aae0204711ab
119151,Foi entregue somente 1. Quero saber do outro p...,ruim ou pessimo,11c177c8e97725db2631073c19f07b62
119152,Foi entregue somente 1. Quero saber do outro p...,ruim ou pessimo,11c177c8e97725db2631073c19f07b62


removendo o que ficou de duplicado, por conta da forma de pagamento

In [121]:
df_titulos_mensagens = df_titulos_mensagens.drop_duplicates(subset=['order_id'])

In [124]:
df_titulos_mensagens.describe()

,new_message_review,new_title_review,order_id
count,99439,99439,99441
unique,36847,4601,99441
top,bom ou otimo,bom ou otimo,0829f68cdf3850bb5977ac321e0fb8d5
freq,49638,67798,1


In [123]:
df_titulos_mensagens.head(10)

,new_message_review,new_title_review,order_id
0,"Não testei o produto ainda, mas ele veio corre...",bom ou otimo,e481f51cbdc54678b7cc49136f2d6af7
3,Muito bom o produto.,Muito boa a loja,53cdb2fc8bc7dce0b6741e2150273451
4,bom ou otimo,bom ou otimo,47770eb9100c2d0c44946d9cf07ec65d
5,O produto foi exatamente o que eu esperava e e...,bom ou otimo,949d5b44dbf5de918fe9c16f97b45f8a
6,bom ou otimo,bom ou otimo,ad21c59c0840e6cb83a9ceb5573f8159
7,bom ou otimo,bom ou otimo,a4591c265e18cb1dcee52889e2d8acc3
8,fiquei triste por n ter me atendido.,ruim ou pessimo,136cce7faa42fdb2cefd53fdc79a6098
9,bom ou otimo,bom ou otimo,6514b8ad8028c9f2cc2374ded245783f
10,ruim ou pessimo,ruim ou pessimo,76c6e866289321a7c93b82b54852dc33
11,bom ou otimo,bom ou otimo,e69bfb5eb88e0ed6a785585b27e16dbf


#SALVANDO O CSV AQUI PARA SEGUIR AMANHÃ

In [125]:
df_titulos_mensagens.to_csv('/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_titulos_mensagens.csv',  index=False, encoding='utf8')

# Carregando os dados que foram salvos acima

In [5]:
dados_tratados = '/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_titulos_mensagens.csv'
ds_dados_tratados = pd.read_csv(dados_tratados, index_col=False, encoding='utf8')
transformacao_final = ds_dados_tratados.copy() # só pra manter o mesmo nome


In [6]:
transformacao_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99442 entries, 0 to 99441
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   new_message_review  99439 non-null  object
 1   new_title_review    99439 non-null  object
 2   order_id            99441 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


In [7]:
transformacao_final['col_tratada'] = None

In [8]:
def ColTratada():
  for index, row in transformacao_final.iterrows():
    if (transformacao_final['new_message_review'][index] != transformacao_final['new_title_review'][index]):
      transformacao_final['col_tratada'][index] = str(transformacao_final['new_message_review'][index]) + ' ' + str(transformacao_final['new_title_review'][index])
    else:
      transformacao_final['col_tratada'][index] = str(transformacao_final['new_message_review'][index])

In [9]:
ColTratada()

In [10]:
transformacao_final.head(5)

,new_message_review,new_title_review,order_id,col_tratada
0,"Não testei o produto ainda, mas ele veio corre...",bom ou otimo,e481f51cbdc54678b7cc49136f2d6af7,"Não testei o produto ainda, mas ele veio corre..."
1,Muito bom o produto.,Muito boa a loja,53cdb2fc8bc7dce0b6741e2150273451,Muito bom o produto. Muito boa a loja
2,bom ou otimo,bom ou otimo,47770eb9100c2d0c44946d9cf07ec65d,bom ou otimo
3,O produto foi exatamente o que eu esperava e e...,bom ou otimo,949d5b44dbf5de918fe9c16f97b45f8a,O produto foi exatamente o que eu esperava e e...
4,bom ou otimo,bom ou otimo,ad21c59c0840e6cb83a9ceb5573f8159,bom ou otimo


In [11]:
df_titulos_mensagens = transformacao_final[['order_id', 'col_tratada']]

In [18]:
#df_titulos_mensagens.to_csv('/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_col_tratada.csv',  index=False, encoding='utf8')
dados_tratados = '/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_col_tratada.csv'
ds_dados_tratados = pd.read_csv(dados_tratados, index_col=False, encoding='utf8')
transformacao_final = ds_dados_tratados.copy() # só pra manter o mesmo nome


In [20]:
transformacao_final['col_tratada'].head(5)

0    Não testei o produto ainda, mas ele veio corre...
1                Muito bom o produto. Muito boa a loja
2                                         bom ou otimo
3    O produto foi exatamente o que eu esperava e e...
4                                         bom ou otimo
Name: col_tratada, dtype: object

# Regular expressions

Limpeza dos textos

In [15]:
import re

## Novas linhas e quebras

In [16]:
# removendo quebras de linha e outros caracteres especiais
def re_breakline(text_list):
    return [re.sub('[\n\r\W]', ' ', r) for r in text_list]

In [21]:
lst_col_tratada = list(transformacao_final['col_tratada'].values)

transformacao_final['col_tratada'] = re_breakline(lst_col_tratada)

In [22]:
transformacao_final['col_tratada'].head(5)

0    Não testei o produto ainda  mas ele veio corre...
1                Muito bom o produto  Muito boa a loja
2                                         bom ou otimo
3    O produto foi exatamente o que eu esperava e e...
4                                         bom ou otimo
Name: col_tratada, dtype: object

## Sites links

In [23]:
def re_hiperlinks(text_list):
    pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return [re.sub(pattern, ' link ', r) for r in text_list]

In [24]:
transformacao_final['col_tratada'] = re_hiperlinks(transformacao_final['col_tratada'])

In [25]:
transformacao_final['col_tratada'].head(5)

0    Não testei o produto ainda  mas ele veio corre...
1                Muito bom o produto  Muito boa a loja
2                                         bom ou otimo
3    O produto foi exatamente o que eu esperava e e...
4                                         bom ou otimo
Name: col_tratada, dtype: object

Removendo espaços em branco

In [33]:
def re_whitespaces(text_list):
    white_spaces = [re.sub('\s+', ' ', r) for r in text_list]
    white_spaces_end = [re.sub('[ \t]+$', '', r) for r in white_spaces]
    return white_spaces_end

In [50]:
lista_textos = re_whitespaces(transformacao_final['col_tratada'])

In [51]:
transformacao_final['col_tratada'] = lista_textos

In [52]:
transformacao_final['col_tratada'].head(5)

0    Não testei o produto ainda mas ele veio corret...
1                 Muito bom o produto Muito boa a loja
2                                         bom ou otimo
3    O produto foi exatamente o que eu esperava e e...
4                                         bom ou otimo
Name: col_tratada, dtype: object

## Datas

In [26]:
def re_dates(text_list):
    pattern = '([0-2][0-9]|(3)[0-1])(\/|\.)(((0)[0-9])|((1)[0-2]))(\/|\.)\d{2,4}'
    return [re.sub(pattern, ' data ', r) for r in text_list]

In [27]:
transformacao_final['col_tratada'] = re_dates(transformacao_final['col_tratada'])

## Moeda

In [28]:
def re_money(text_list):
    pattern = '[R]{0,1}\$[ ]{0,}\d+(,|\.)\d+'
    return [re.sub(pattern, ' dinheiros ', r) for r in text_list]

In [29]:
transformacao_final['col_tratada'] = re_money(transformacao_final['col_tratada'])

In [30]:
transformacao_final['col_tratada'].head(4)

0    Não testei o produto ainda  mas ele veio corre...
1                Muito bom o produto  Muito boa a loja
2                                         bom ou otimo
3    O produto foi exatamente o que eu esperava e e...
Name: col_tratada, dtype: object

## Números

In [31]:
def re_numbers(text_list):
    return [re.sub('[0-9]+', ' numero ', r) for r in text_list]

In [32]:
transformacao_final['col_tratada'] = re_numbers(transformacao_final['col_tratada'])

## Negação

In [33]:
def re_negation(text_list):
    return [re.sub('([nN][ãÃaA][oO]|[ñÑ]| [nN] )', ' negação ', r) for r in text_list]

In [34]:
transformacao_final['col_tratada'] = re_negation(transformacao_final['col_tratada'])

In [35]:
transformacao_final['col_tratada'].head(4)

0     negação  testei o produto ainda  mas ele veio...
1                Muito bom o produto  Muito boa a loja
2                                         bom ou otimo
3    O produto foi exatamente o que eu esperava e e...
Name: col_tratada, dtype: object

## Espaço em branco inicio e fim de frase

In [36]:
def re_whitespaces(text_list):
    white_spaces = [re.sub('\s+', ' ', r) for r in text_list]
    white_spaces_end = [re.sub('[ \t]+$', '', r) for r in white_spaces]
    return white_spaces_end

In [37]:
transformacao_final['col_tratada'] = re_whitespaces(transformacao_final['col_tratada'])

In [38]:
transformacao_final['col_tratada'].head(4)

0     negação testei o produto ainda mas ele veio c...
1                 Muito bom o produto Muito boa a loja
2                                         bom ou otimo
3    O produto foi exatamente o que eu esperava e e...
Name: col_tratada, dtype: object

## Stopwords

In [39]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [50]:
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

teste

In [41]:
pt_stopwords = stopwords.words('portuguese')
print(f'Total portuguese stopwords in the nltk.corpous module: {len(pt_stopwords)}')
pt_stopwords[:10]

Total portuguese stopwords in the nltk.corpous module: 204


['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um']

fim do teste

In [43]:
# Defining a function to remove the stopwords and to lower the comments
def stopwords_removal(text, cached_stopwords=stopwords.words('portuguese')):
    return [c.lower() for c in text.split() if c.lower() not in cached_stopwords]

In [45]:
lst_col_tratada1 = list(transformacao_final['col_tratada'].values)

transformacao_final['col_tratada'] = [' '.join(stopwords_removal(lst_col_tratada1)) for lst_col_tratada1 in transformacao_final['col_tratada'].values]

In [46]:
transformacao_final['col_tratada'].head(5)

0    negação testei produto ainda veio correto boas...
1                                 bom produto boa loja
2                                            bom otimo
3    produto exatamente esperava descrito site cheg...
4                                            bom otimo
Name: col_tratada, dtype: object

## Stemming 

In [52]:
 import nltk ## caso inicie daqui
 nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [53]:
def stemming_process(text, stemmer=RSLPStemmer()):    
    return [stemmer.stem(c) for c in text.split()]

In [54]:
transformacao_final['col_tratada'] = [' '.join(stemming_process(lst_col_tratada1)) for lst_col_tratada1 in transformacao_final['col_tratada'].values ]

In [55]:
transformacao_final['col_tratada'].head(5)

0    neg test produt aind vei corret boa cond apen ...
1                                   bom produt boa loj
2                                             bom otim
3    produt exat esper descrit sit cheg bem ant dat...
4                                             bom otim
Name: col_tratada, dtype: object

# Salvando dia 13/01
dados_transformados_final.csv

In [56]:
transformacao_final.to_csv('/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_final.csv', index=False, encoding='utf8')
#dados_tratados = '/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_col_tratada.csv'
#ds_dados_tratados = pd.read_csv(dados_tratados, index_col=False, encoding='utf8')
#transformacao_final = ds_dados_tratados.copy() # só pra manter o mesmo nome

# a partir daqui a noite vamos começar a brincadeira